# Chen et al. (2019)

## Turnover rate of ${}^{14}C$, ${}^{15}N$ and ${}^{33}P$

### Importing functions

In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution

### This is the DEB model

In [2]:
def DEBmodel (y, t, pars):
    #define initial pools
    e=y[0];    X1=y[1]
    
    #define parameters
    #yA=pars[0]; 
    #Km=pars[1];     
    v=pars;
    m=0.03802298; 
    g=0.89960365; 
    #k=pars[1];
    ce=3.23343743;
    MX1=ce/4;
    
    #Scaling function for substrate uptake
    #f=S/(Km+S)
    #Fluxes
    #uptake=(v*ce/yA)*X1*f
    growth = (v*e-m*g)/(e + g)
    
    #Define derivatives
    #dSdt = -uptake
    dedt = -v*e
    dX1dt = growth*X1 #- k*X1
    #dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1 
           
    return dedt, dX1dt;

### The function below uses the output from DEBmodel to convert biomass pools $e$ and $X_{1}$ to microbial biomass ($B$) and the isotopes in chloroform flush 

In [3]:
def calcDEB (model, pars, t, y0):
    #model parameters
    ##v
    pars_model=pars[0]
    #conversion factors
    ##CNX1, CNe, CPX1, CPe
    conversions=pars[1:5]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate biomass (B), total DNA, and flush (Flush)
    Bc=(3.23343743/4 + 3.23343743*y[:, 0])*y[:, 1]
    Bn=(3.23343743/4/conversions[0] + 3.23343743*y[:, 0]/conversions[1])*y[:, 1]
    Bp=(3.23343743/4/conversions[2] + 3.23343743*y[:, 0]/conversions[3])*y[:, 1]
    C14 = (0.25108916/4 + y[:, 0])/(0.25 + y[:, 0])*Bc
    N15 = (0.25108916/4 + y[:, 0])/(0.25 + y[:, 0])*Bn
    P33 = (0.25108916/4 + y[:, 0])/(0.25 + y[:, 0])*Bp    
    
    #Create data with predictions
    yhat = np.concatenate((C14.reshape(len(d.Time),1),
                           N15.reshape(len(d.Time),1),
                           P33.reshape(len(d.Time),1)), axis=1)

    return yhat

### Objective function is defined

In [4]:
def obj_funDEB (x):
    #define parameters
    ##v, CNe, CNX1, CPe, CPX1, e0
    pars = x

    #initial conditions
    e_i = pars[5]
    X1_i = d.C14[0]/((0.25*0.25108916+pars[5])*3.23343743)
    
    y0 = np.array([e_i, X1_i])

    #times
    t = d.Time-4

    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)

    #observations
    obs=np.concatenate((np.array([d.C14]).reshape(len(d.Time),1),
                        np.array([d.N15]).reshape(len(d.Time),1),
                        np.array([d.P33]).reshape(len(d.Time),1)),
                     axis=1)

    #weights
    weights=np.concatenate((np.nanstd(d.C14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.N15).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.P33)).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)

    out=np.nansum(((yhat_full-obs)/weights)**2)

    return out

### Goodness of fit is calculated

In [5]:
def goodnessDEB (x):
    #define parameters
    ##v, CNe, CNX1, CPe, CPX1, e0
    pars = x

    #initial conditions
    e_i = pars[5]
    X1_i = d.C14[0]/((0.25*0.25108916+pars[5])*3.23343743)
    
    y0 = np.array([e_i, X1_i])

    #times
    t = d.Time-4

    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)

    #observations
    obs=np.concatenate((np.array([d.C14]).reshape(len(d.Time),1),
                        np.array([d.N15]).reshape(len(d.Time),1),
                        np.array([d.P33]).reshape(len(d.Time),1)),
                     axis=1)
    
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    out = np.array([R2, ll, AIC])

    return out

In [6]:
def predDEB (model, pars, t, y0):
    #model parameters
    ##v
    pars_model=pars[0]
    #conversion factors
    ##CNX1, CNe, CPX1, CPe
    conversions=pars[1:5]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate biomass (B), total DNA, and flush (Flush)
    Bc=(3.23343743/4 + 3.23343743*y[:, 0])*y[:, 1]
    Bn=(3.23343743/4/conversions[0] + 3.23343743*y[:, 0]/conversions[1])*y[:, 1]
    Bp=(3.23343743/4/conversions[2] + 3.23343743*y[:, 0]/conversions[3])*y[:, 1]
    C14 = (0.25108916/4 + y[:, 0])/(0.25 + y[:, 0])*Bc
    N15 = (0.25108916/4 + y[:, 0])/(0.25 + y[:, 0])*Bn
    P33 = (0.25108916/4 + y[:, 0])/(0.25 + y[:, 0])*Bp   
    
    #Create data with predictions
    yhat = np.concatenate((C14.reshape(len(np.arange(0, 13.5, 0.5)),1),
                           N15.reshape(len(np.arange(0, 13.5, 0.5)),1),
                           P33.reshape(len(np.arange(0, 13.5, 0.5)),1)), axis=1)

    return yhat

In [7]:
#reading the data
dall = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/Chen2019.csv', sep=',')
print(dall)

                Study  Time        C14       N15       P33 Treatment
0  Chen et al. (2019)     4   2.096003       NaN  0.024927      LowP
1  Chen et al. (2019)     6   0.679309  0.075931  0.005554      LowP
2  Chen et al. (2019)    11   0.387052  0.041242  0.004391      LowP
3  Chen et al. (2019)    17   0.352206  0.028137  0.005554      LowP
4  Chen et al. (2019)     4  16.959551  2.126852  0.936778     HighP
5  Chen et al. (2019)     6  13.318860  1.995418  0.531740     HighP
6  Chen et al. (2019)    11  10.891733  1.229936  0.420665     HighP
7  Chen et al. (2019)    17   9.981561  0.781670  0.470132     HighP


### Low P soil

In [8]:
d = dall[dall.Treatment == "LowP"]
d = d.reset_index()
print(d)

   index               Study  Time       C14       N15       P33 Treatment
0      0  Chen et al. (2019)     4  2.096003       NaN  0.024927      LowP
1      1  Chen et al. (2019)     6  0.679309  0.075931  0.005554      LowP
2      2  Chen et al. (2019)    11  0.387052  0.041242  0.004391      LowP
3      3  Chen et al. (2019)    17  0.352206  0.028137  0.005554      LowP


In [9]:
chen_lowP=dual_annealing(obj_funDEB, [(0.01, 20), #v
                                             #(1e-12, 0.1), #m
                                             #(0.1, 3), #g
                                             #(1e-12, 0.1), #k
                                             #(0.1, 10), #ce
                                             (0.1, 100), #CNe
                                             (0.1, 100), #CNX1
                                             (0.1, 500), #CPe
                                             (0.1, 500), #CPX1                                            
                                             #(0, 1), #nX1
                                             (0, 1)]) #e_0

In [10]:
print(chen_lowP)
print(goodnessDEB(chen_lowP.x))

     fun: 0.1065129136310129
 message: ['Maximum number of iteration reached']
    nfev: 14892
    nhev: 0
     nit: 1000
    njev: 413
  status: 0
 success: True
       x: array([ 1.69024135, 11.78192523,  0.67022326, 96.69631257, 72.83456438,
        0.20931707])
[ 0.99749582 -0.01377298 12.02754597]


In [11]:
chen_lowPDE=differential_evolution(obj_funDEB, [(0.01, 20), #v
                                             #(1e-12, 0.1), #m
                                             #(0.1, 3), #g
                                             #(1e-12, 0.1), #k
                                             #(0.1, 10), #ce
                                             (0.1, 100), #CNe
                                             (0.1, 100), #CNX1
                                             (0.1, 500), #CPe
                                             (0.1, 500), #CPX1                                            
                                             #(0, 1), #nX1
                                             (0, 1)]) #e_0

In [12]:
print(chen_lowPDE)
print(goodnessDEB(chen_lowPDE.x))

     fun: 0.10651754712646785
     jac: array([ 6.99995621e-06, -8.67778000e-06, -2.70061749e-06,  1.42524970e-05,
        5.31339195e-05, -4.96325203e-05])
 message: 'Optimization terminated successfully.'
    nfev: 11536
     nit: 125
 success: True
       x: array([ 1.69007729, 11.76682922,  0.67142073, 96.47618152, 73.06729149,
        0.20977921])
[ 0.99749869 -0.01375722 12.02751445]


In [13]:
#np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Marstorp1999Pars.csv', marstorp_debparsDE.x.reshape(1,7), delimiter=",")

## Solution over time for visualization in R

In [14]:
#initial conditions
e_i = chen_lowPDE.x[5]
X1_i = d.C14[0]/((0.25*0.25108916+chen_lowPDE.x[5])*3.23343743)
    
y0 = np.array([e_i, X1_i])
    
#times
t = np.arange(0, 13.5, 0.5)

#model simulations
ChenLowPPred = predDEB(DEBmodel, chen_lowPDE.x, t, y0)
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/ChenLowPPred.csv', ChenLowPPred, delimiter=",")

### High P soil

In [15]:
d = dall[dall.Treatment == "HighP"]
d = d.reset_index()
print(d)

   index               Study  Time        C14       N15       P33 Treatment
0      4  Chen et al. (2019)     4  16.959551  2.126852  0.936778     HighP
1      5  Chen et al. (2019)     6  13.318860  1.995418  0.531740     HighP
2      6  Chen et al. (2019)    11  10.891733  1.229936  0.420665     HighP
3      7  Chen et al. (2019)    17   9.981561  0.781670  0.470132     HighP


In [16]:
chen_highP=dual_annealing(obj_funDEB, [(0.01, 20), #v
                                             #(1e-12, 0.1), #m
                                             #(0.1, 3), #g
                                             #(1e-12, 0.1), #k
                                             #(0.1, 10), #ce
                                             (0.1, 100), #CNe
                                             (0.1, 100), #CNX1
                                             (0.1, 500), #CPe
                                             (0.1, 500), #CPX1                                            
                                             #(0, 1), #nX1
                                             (0, 1)]) #e_0

In [17]:
print(chen_highP)
print(goodnessDEB(chen_highP.x))

     fun: 1.2323937239310514
 message: ['Maximum number of iteration reached']
    nfev: 17790
    nhev: 0
     nit: 1000
    njev: 827
  status: 0
 success: True
       x: array([1.89375706e+00, 8.21168325e+00, 4.52251949e+00, 2.43833087e+01,
       2.54106471e+00, 1.04823794e-02])
[ 0.99616283 -0.02302304 12.04604608]


In [18]:
chen_highPDE=differential_evolution(obj_funDEB, [(0.01, 20), #v
                                             #(1e-12, 0.1), #m
                                             #(0.1, 3), #g
                                             #(1e-12, 0.1), #k
                                             #(0.1, 10), #ce
                                             (0.1, 100), #CNe
                                             (0.1, 100), #CNX1
                                             (0.1, 500), #CPe
                                             (0.1, 500), #CPX1                                            
                                             #(0, 1), #nX1
                                             (0, 1)]) #e_0

In [19]:
print(chen_highPDE)
print(goodnessDEB(chen_highPDE.x))

     fun: 1.2333844008981423
     jac: array([-1.86517469e-06,  1.09112710e-04,  1.19302347e-03,  1.58282263e-03,
       -7.39608379e-04,  3.15516502e-02])
 message: 'Optimization terminated successfully.'
    nfev: 5356
     nit: 54
 success: True
       x: array([6.08433698e+00, 8.16609547e+00, 5.41477623e+00, 2.43196053e+01,
       2.53223960e+00, 1.03967067e-02])
[ 0.99623405 -0.02259569 12.04519138]


## Solution over time for visualization in R

In [20]:
#initial conditions
e_i = chen_highPDE.x[5]
X1_i = d.C14[0]/((0.25*0.25108916+chen_highPDE.x[5])*3.23343743)
    
y0 = np.array([e_i, X1_i])
    
#times
t = np.arange(0, 13.5, 0.5)

#model simulations
ChenHighPPred = predDEB(DEBmodel, chen_highPDE.x, t, y0)
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/ChenHighPPred.csv', ChenHighPPred, delimiter=",")